In [1]:
import itertools
import numpy
import time
import pypot
import pypot.dynamixel

ports = pypot.dynamixel.get_available_ports()

if not ports:
    raise IOError('No port available.')

port = ports[0]
dxl_io = pypot.dynamixel.DxlIO(port)
found_ids = dxl_io.scan()
ids = found_ids[:2]
dxl_io.enable_torque(ids)

OSError: No port available.

In [3]:
AMP = 1
FREQ = 0.5

speed = dict(zip(ids, itertools.repeat(10)))
dxl_io.set_moving_speed(speed)

pos = dict(zip(ids, itertools.repeat(0)))
dxl_io.set_goal_position(pos)

t0 = time.time()
while True:
    t = time.time()
    if (t - t0) > 5:
        break

    pos = AMP * numpy.sin(2 * numpy.pi * FREQ * t)
    dxl_io.set_goal_position(dict(zip(ids, itertools.repeat(pos))))

    time.sleep(0.02)

In [17]:
from IPython.display import display, clear_output

while True :
    i = dxl_io.get_present_position((ids[0],))
    print('\r', 'Iteration : ', i, end='')
    time.sleep(0.1)
    print('', end='\r')

 Iteration :  (-46.11,))

KeyboardInterrupt: 

In [3]:
######
#TEST#
######

import numpy as np
import cv2
from primesense import openni2
from primesense import _openni2 as c_api



##Path to rgb save directory
path_rgb = "save_files/rgb/"
##Path to depth save directory
path_depth = "save_files/depth/"

## Path of the OpenNI redistribution OpenNI2.so
dist ='AstraSDK/lib/Plugins/openni2'

## initialize openni and check
openni2.initialize(dist) #'C:\Program Files\OpenNI2\Redist\OpenNI2.dll') # accepts the path of the OpenNI redistribution
if (openni2.is_initialized()):
    print ("openNI2 initialized")
else:
    print ("openNI2 not initialized")

## Register the device
dev = openni2.Device.open_any()

## create the streams stream
rgb_stream = dev.create_color_stream()
depth_stream = dev.create_depth_stream()


## Check and configure the mirroring -- default is True
# print 'Mirroring info1', depth_stream.get_mirroring_enabled()
depth_stream.set_mirroring_enabled(False)
rgb_stream.set_mirroring_enabled(False)


## start the stream
rgb_stream.start()
depth_stream.start()

## synchronize the streams
dev.set_depth_color_sync_enabled(True) # synchronize the streams

## IMPORTANT: ALIGN DEPTH2RGB (depth wrapped to match rgb stream)
dev.set_image_registration_mode(openni2.IMAGE_REGISTRATION_DEPTH_TO_COLOR)

##help(dev.set_image_registration_mode)


def get_rgb():
    """
    Returns numpy 3L ndarray to represent the rgb image.
    """
    bgr   = np.fromstring(rgb_stream.read_frame().get_buffer_as_uint8(),dtype=np.uint8).reshape(480, 640,3)
    rgb   = cv2.cvtColor(bgr,cv2.COLOR_BGR2RGB)
    return rgb    
#get_rgb

def get_depth():
    """
    Returns numpy ndarrays representing the raw and ranged depth images.
    Outputs:
        dmap:= distancemap in mm, 1L ndarray, dtype=uint16, min=0, max=2**12-1
        d4d := depth for dislay, 3L ndarray, dtype=uint8, min=0, max=255    
    Note1: 
        fromstring is faster than asarray or frombuffer
    Note2:     
        .reshape(120,160) #smaller image for faster response 
                OMAP/ARM default video configuration
        .reshape(240,320) # Used to MATCH RGB Image (OMAP/ARM)
                Requires .set_video_mode
    """
    dmap = np.fromstring(depth_stream.read_frame().get_buffer_as_uint16(),dtype=np.uint16).reshape(480, 640)  # Works & It's FAST
    d4d = np.uint8(dmap.astype(float) *255/ 2**12-1) # Correct the range. Depth images are 12bits
    d4d = 255 - cv2.cvtColor(d4d,cv2.COLOR_GRAY2RGB)
    return dmap, d4d
#get_depth




def mask_rgbd(d4d,rgb, th=0):
    """
    Overlays images and uses some blur to slightly smooth the mask
    (3L ndarray, 3L ndarray) -> 3L ndarray
    th:= threshold
    """
    mask = d4d.copy()
    #mask = cv2.GaussianBlur(mask, (5,5),0)
    idx =(mask>th)
    mask[idx] = rgb[idx]
    return mask
#mask_rgbd


## main loop
s=0
done = False
while not done:
    key = cv2.waitKey(1) & 255
    ## Read keystrokes
    if key == 27: # terminate
        print ("\tESC key detected!")
        done = True
    elif chr(key) =='s': #screen capture
        print ("\ts key detected. Saving image and distance map {}".format(s))
        cv2.imwrite("" + path_rgb + "rgb_"+str(s)+'.png', rgb)
        cv2.imwrite("" + path_depth + "depth_"+str(s)+'.png',dmap)
        s+=1
    #if
    
    ## Streams
    #RGB
    rgb = get_rgb()
    
    #DEPTH
    dmap,d4d = get_depth()
    
    # Overlay rgb over the depth stream
    rgbd  = mask_rgbd(d4d,rgb)
    
    # canvas
    canvas = np.hstack((d4d,rgb,rgbd))
    
    ## Distance map
    #print ('Center pixel is {} mm away'.format(dmap[119,159]))

    ## Display the stream
    cv2.imshow('depth || rgb || rgbd', canvas )
# end while

## Release resources 
cv2.destroyAllWindows()
rgb_stream.stop()
depth_stream.stop()
openni2.unload()
print ("Terminated")

openNI2 initialized


/home/jeremy/.local/lib/python3.5/site-packages/ipykernel_launcher.py:58: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/home/jeremy/.local/lib/python3.5/site-packages/ipykernel_launcher.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


	s key detected. Saving image and distance map 0
	s key detected. Saving image and distance map 1
	s key detected. Saving image and distance map 2
	s key detected. Saving image and distance map 3
	s key detected. Saving image and distance map 4
	s key detected. Saving image and distance map 5
	s key detected. Saving image and distance map 6
	ESC key detected!
Terminated


In [4]:
######
#TEST#
######

import numpy as np
import cv2
from primesense import openni2
from primesense import _openni2 as c_api



##Path to rgb save directory
path_rgb = "save_files/scene/rgb/"
##Path to depth save directory
path_depth = "save_files/scene/depth/"

## Path of the OpenNI redistribution OpenNI2.so
dist ='AstraSDK/lib/Plugins/openni2'

## initialize openni and check
openni2.initialize(dist) #'C:\Program Files\OpenNI2\Redist\OpenNI2.dll') # accepts the path of the OpenNI redistribution
if (openni2.is_initialized()):
    print ("openNI2 initialized")
else:
    print ("openNI2 not initialized")

## Register the device
dev = openni2.Device.open_any()

## create the streams stream
rgb_stream = dev.create_color_stream()
rgb_stream.set_video_mode(c_api.OniVideoMode(pixelFormat = c_api.OniPixelFormat.ONI_PIXEL_FORMAT_RGB888, resolutionX = 1280, resolutionY = 960, fps = 30))
rgb_stream.set_property(c_api.ONI_STREAM_PROPERTY_AUTO_EXPOSURE, True)

## Check and configure the mirroring -- default is True
# print 'Mirroring info1', depth_stream.get_mirroring_enabled()
rgb_stream.set_mirroring_enabled(False)


## start the stream
rgb_stream.start()

def get_rgb():
    """
    Returns numpy 3L ndarray to represent the rgb image.
    """
    bgr   = np.fromstring(rgb_stream.read_frame().get_buffer_as_uint8(),dtype=np.uint8).reshape(960, 1280,3)
    rgb   = cv2.cvtColor(bgr,cv2.COLOR_BGR2RGB)
    return rgb    


s=0
done = False
while not done:
    key = cv2.waitKey(1) & 255
    ## Read keystrokes
    if key == 27: # terminate
        print ("\tESC key detected!")
        done = True
    elif chr(key) =='s': #screen capture
        print ("\ts key detected. Saving image and distance map {}".format(s))
        cv2.imwrite("" + path_rgb + "rgb_"+str(s)+'.png', rgb)
        s+=1
    #if
    
    ## Streams
    #RGB
    rgb = get_rgb()

    # canvas
    canvas = np.hstack((rgb,rgb))
    
    ## Distance map
    #print ('Center pixel is {} mm away'.format(dmap[119,159]))

    ## Display the stream
    cv2.imshow('rgb',rgb )
# end while

## Release resources 
cv2.destroyAllWindows()
rgb_stream.stop()
depth_stream.stop()
openni2.unload()
print ("Terminated")

openNI2 initialized


/home/jeremy/.local/lib/python3.5/site-packages/ipykernel_launcher.py:47: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


	ESC key detected!


ArgumentError: argument 1: <class 'TypeError'>: expected LP__OniStream instance instead of ClosedHandle